# Viasualisation of the data labels

In [1]:
import pandas as pd

## Celeba labels2.csv data

In [2]:
celeba_df = pd.read_csv('./datasets/celeba/labels2.csv')
celeba_df = celeba_df.drop('Unnamed: 0', axis = 1)
celeba_df.head()

,img_name,gender,smiling
0,0.jpg,-1,1
1,1.jpg,-1,1
2,2.jpg,1,-1
3,3.jpg,-1,-1
4,4.jpg,-1,-1


In [3]:
celeba_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   img_name  5000 non-null   object
 1   gender    5000 non-null   int64 
 2   smiling   5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


In [4]:
celeba_df.isnull().sum()

img_name    0
gender      0
smiling     0
dtype: int64

In [5]:
celeba_df.value_counts('gender')

gender
 1    2500
-1    2500
dtype: int64

In [6]:
celeba_df.value_counts('smiling')

smiling
 1    2500
-1    2500
dtype: int64

## Cartoon_set labels.scv data

In [7]:
cartoon_df = pd.read_csv('./datasets/cartoon_set/labels.csv')
cartoon_df = cartoon_df.drop('Unnamed: 0', axis = 1)
cartoon_df.head()

,eye_color,face_shape,file_name
0,1,4,0.png
1,2,4,1.png
2,2,3,2.png
3,2,0,3.png
4,0,2,4.png


In [8]:
cartoon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   eye_color   10000 non-null  int64 
 1   face_shape  10000 non-null  int64 
 2   file_name   10000 non-null  object
dtypes: int64(2), object(1)
memory usage: 234.5+ KB


In [9]:
cartoon_df.isnull().sum()

eye_color     0
face_shape    0
file_name     0
dtype: int64

In [10]:
cartoon_df.value_counts('face_shape')

face_shape
4    2000
3    2000
2    2000
1    2000
0    2000
dtype: int64

In [11]:
cartoon_df.value_counts('eye_color')

eye_color
1    2018
4    2017
0    2004
3    1992
2    1969
dtype: int64

# Import Required Libraries

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2
import dlib
import tensorflow 
from keras.preprocessing import image
from sklearn.metrics import classification_report,accuracy_score
from sklearn import svm

/opt/anaconda3/envs/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/python36/lib/python3.6/site-packages/tensorf

## Define some useful functions

In [13]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [14]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

In [15]:
def run_dlib_shape(image):
    # in this function we load the image, detect the landmarks of the face, and then return the image and the landmarks
    # load the input image, resize it, and convert it to grayscale
    resized_image = image.astype('uint8')

    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    gray = gray.astype('uint8')

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    num_faces = len(rects)

    if num_faces == 0:
        return None, resized_image

    face_areas = np.zeros((1, num_faces))
    face_shapes = np.zeros((136, num_faces), dtype=np.int64) ## 5000 lines

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        temp_shape = predictor(gray, rect)
        temp_shape = shape_to_np(temp_shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)],
        #   (x, y, w, h) = face_utils.rect_to_bb(rect)
        (x, y, w, h) = rect_to_bb(rect)
        face_shapes[:, i] = np.reshape(temp_shape, [136])
        face_areas[0, i] = w * h
    # find largest face and keep
    dlibout = np.reshape(np.transpose(face_shapes[:, np.argmax(face_areas)]), [68, 2])

    return dlibout, resized_image

# Tasks A

Let's create a path to all the images using the os library

In [16]:
global basedir, image_paths, target_size
basedir = './datasets/celeba'
images_dir = os.path.join(basedir,'img')
labels_filename = 'labels2.csv'

For this task we will use the Facial Landmark approchach. "The aim is to find frontal human faces in an image and estimate their pose. The pose takes the form of 68 landmarks. These are points on the face such as the corners of the mouth, along the eyebrows, on the eyes, and so forth." (http://dlib.net/face_landmark_detection.py.html)

In [17]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

## Task A1: Gender Detection

In [18]:
def extract_features_labels_gender():
    
    image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)] # Créer un path en incluant le nom de chaque mage ?
    target_size = None
    labels_file = open(os.path.join(basedir, labels_filename), 'r')
    lines = labels_file.readlines()
    gender_labels = {line.split(',')[0] : int(line.split(',')[2]) for line in lines[1:]}
    if os.path.isdir(images_dir): # SI le Path existe 
        all_features = []
        all_labels = []
        for img_path in image_paths: # Pour chaque image 
            file_name= img_path.split('.')[1].split('/')[-1]

            # load image
            img = image.img_to_array(
                image.load_img(img_path,
                               target_size=target_size,
                               interpolation='bicubic'))
            features, _ = run_dlib_shape(img)
            if features is not None:
                all_features.append(features)
                all_labels.append(gender_labels[file_name])

    landmark_features = np.array(all_features)
    gender_labels = (np.array(all_labels) + 1)/2 # simply converts the -1 into 0, so male=0 and female=1
    return landmark_features, gender_labels

## Linear SVC:

We will exctract the features from the pictures and the gender value from the csv file (with 0 instead of -1 for the male), using the extract_features_labels() function and separate these features into a training and testing set

In [19]:
def get_data():

    X, y = extract_features_labels_gender()
    Y = np.array([y, -(y - 1)]).T
    tr_X = X[:3500]
    tr_Y = Y[:3500]
    te_X = X[3500:]
    te_Y = Y[3500:]

    return tr_X, tr_Y, te_X, te_Y

In [20]:
tr_X, tr_Y, te_X, te_Y= get_data()

In [23]:
def img_SVM(training_images, training_labels, test_images, test_labels):
    classifier = svm.SVC(kernel = 'linear')
    classifier.fit(training_images, training_labels)
    pred_gender = classifier.predict(test_images)
    print("Accuracy:", accuracy_score(test_labels, pred_gender))

    print(pred_gender)
    return pred_gender

pred_gender=img_SVM(tr_X.reshape((3500, 68*2)), list(zip(*tr_Y))[0], te_X.reshape((1300, 68*2)), list(zip(*te_Y))[0])


Accuracy: 0.9292307692307692
[1. 1. 0. ... 0. 1. 0.]


## Task A2: Smiling Detection

In [24]:
def extract_features_labels_smile():
    
    image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)] # Créer un path en incluant le nom de chaque mage ?
    target_size = None
    labels_file = open(os.path.join(basedir, labels_filename), 'r')
    lines = labels_file.readlines()
    smile_labels = {line.split(',')[0] : int(line.split(',')[3]) for line in lines[1:]}
    if os.path.isdir(images_dir): # SI le Path existe 
        all_features = []
        all_labels = []
        for img_path in image_paths: # Pour chaque image 
            file_name= img_path.split('.')[1].split('/')[-1]

            # load image
            img = image.img_to_array(
                image.load_img(img_path,
                               target_size=target_size,
                               interpolation='bicubic'))
            features, _ = run_dlib_shape(img)
            if features is not None:
                all_features.append(features)
                all_labels.append(smile_labels[file_name])

    landmark_features = np.array(all_features)
    smile_labels = (np.array(all_labels) + 1)/2 # simply converts the -1 into 0, so not smile=0 and smile=1
    return landmark_features, smile_labels

## Linear SVC:

We will exctract the features from the pictures and the gender value from the csv file (with 0 instead of -1 for the "not smile"), using the extract_features_labels() function and separate these features into a training and testing set.

In [25]:
def get_data():

    X, y = extract_features_labels_smile()
    Y = np.array([y, -(y - 1)]).T
    tr_X = X[:3500]
    tr_Y = Y[:3500]
    te_X = X[3500:]
    te_Y = Y[3500:]

    return tr_X, tr_Y, te_X, te_Y

In [26]:
tr_X, tr_Y, te_X, te_Y= get_data()

In [27]:
def img_SVM(training_images, training_labels, test_images, test_labels):
    classifier = svm.SVC(kernel = 'linear')
    classifier.fit(training_images, training_labels)
    pred_smile = classifier.predict(test_images)
    print("Accuracy:", accuracy_score(test_labels, pred_smile))

    print(pred_smile)
    return pred_smile

pred_smile=img_SVM(tr_X.reshape((3500, 68*2)), list(zip(*tr_Y))[0], te_X.reshape((1300, 68*2)), list(zip(*te_Y))[0])


Accuracy: 0.8923076923076924
[1. 1. 1. ... 0. 1. 0.]


# Task B1: Face Shape

Here we want to restore our previously trained CNN model vailable in the B1 file. The model name is "model"

In [ ]:
def restore_net():
    # restore entire net1 to net2
    net2 = torch.load('net.pkl')
    prediction = net2(x)

# Print out results

In [ ]:
print('TA1:{},{};TA2:{},{};TB1:{},{};TB2:{},{};'.format(acc_A1_train, acc_A1_test,
                                                        acc_A2_train, acc_A2_test,
                                                        acc_B1_train, acc_B1_test,
                                                        acc_B2_train, acc_B2_test))